In [1]:
import torch
from utils import DataLoader, evaluate_model
from models import LinearRegressionModel
from matplotlib import pyplot as plt

In [2]:
import logging
logging.basicConfig(level=logging.INFO)

In [3]:
dl = DataLoader(
    benchmark="benchmark_1",
    split=None,
    embedding="bert_cls", 
    max_len=16,
    include_axes=True,
    shuffle_axes=False,  
    normalization="zscore"
)

INFO:utils.dataloader:Initializing DataLoader...
INFO:utils.dataloader:Using device: cuda
INFO:utils.dataloader:Loading dataset for benchmark benchmark_1 with split None
INFO:utils.dataloader:No split specified; loading all splits (train, validation, test).
INFO:utils.dataloader:Dataset loaded successfully.
INFO:utils.dataloader:Initializing embedding for method bert_cls...
INFO:utils.dataloader:Initialized BERT model and tokenizer.
INFO:utils.dataloader:Embedding resources initialized.


In [4]:
data = dl.preprocess()
X_train, y_train = data["train"]
X_valid, y_valid = data["validation"]
X_test, y_test = data["test"]

INFO:utils.dataloader:Starting preprocessing of dataset(s)...
INFO:utils.dataloader:Processing split: train with 878 samples (shuffle_axes=False)...
INFO:utils.dataloader:Processing a single dataset split...
INFO:utils.dataloader:Combined feature vector shape: torch.Size([878, 6912])
INFO:utils.dataloader:Extracting and normalizing target values...
INFO:utils.dataloader:Converting target values to logarithmic scale...
INFO:utils.dataloader:No conversion needed; log_scale is already 1.1.
INFO:utils.dataloader:Normalizing target values...
INFO:utils.dataloader:Z-score parameters: mean=[ 5.44271868e+01  1.15348462e+01 -2.48804100e-02], std=[20.1219028   3.81394874  1.39502752]
INFO:utils.dataloader:Processing split: validation with 247 samples (shuffle_axes=False)...
INFO:utils.dataloader:Processing a single dataset split...
INFO:utils.dataloader:Combined feature vector shape: torch.Size([247, 6912])
INFO:utils.dataloader:Extracting and normalizing target values...
INFO:utils.dataloader:C

In [5]:
print("Shapes:", X_train.shape, y_train.shape)

if isinstance(X_train, torch.Tensor):
    X_train = X_train.cpu().numpy()
if isinstance(X_valid, torch.Tensor):
    X_valid = X_valid.cpu().numpy()
if isinstance(X_test, torch.Tensor):
    X_test = X_test.cpu().numpy()

Shapes: torch.Size([878, 6912]) (878, 3)


In [6]:
grid_params = {
    "estimator__fit_intercept": [True, False]
}

In [7]:
lr_model = LinearRegressionModel()
lr_model.build_model()

INFO:models.linear_regression:Linear Regression model built (wrapped in MultiOutputRegressor).


In [8]:
best_model, best_params = lr_model.grid_search(X_train, y_train, X_valid, y_valid, param_grid=grid_params, cv=5)

print("Best Hyperparameters:", best_params)

INFO:models.linear_regression:Starting grid search for Linear Regression...


Fitting 5 folds for each of 2 candidates, totalling 10 fits


INFO:models.linear_regression:Grid search complete.
INFO:models.linear_regression:Best Hyperparameters: {'estimator__fit_intercept': False}
INFO:models.linear_regression:Validation MSE for best model: 1.4148


Best Hyperparameters: {'estimator__fit_intercept': False}


In [9]:
test_preds = best_model.model.predict(X_test)
mse, mae, r2, nll, crps, pearson_corr, spearman_corr = evaluate_model(y_test, test_preds)
print("Test Performance:")
print(f"MSE: {mse:.4f}, MAE: {mae:.4f}, R2: {r2:.4f}, NLL: {nll:.4f}, CRPS: {crps:.4f}, Pearson: {pearson_corr:.4f}, Spearman: {spearman_corr:.4f}")

Test Performance:
MSE: 1.3610, MAE: 0.9179, R2: -0.3610, NLL: 1.5730, CRPS: 0.9179, Pearson: 0.2047, Spearman: 0.2338


In [10]:
save_path = "baseline_weights/benchmark_1/linear_regression.pkl"
best_model.save(save_path)

INFO:models.linear_regression:Linear Regression model saved to baseline_weights/benchmark_1/linear_regression.pkl
